In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
X = 2 * np.random.rand(100, 1)
y = 4 + 3*X + np.random.rand(100, 1)

In [ ]:
plt.plot(X, y, 'bo',  markersize=3)

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]
print(X_b.shape)

In [ ]:
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

In [ ]:
y_pred = X_b.dot(theta_best)

In [ ]:
plt.plot(X, y, 'bo',  markersize=3)
plt.plot(X, y_pred, 'r-', linewidth=0.5)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

The LinearRegression class is based on the scipy.linalg.lstsq() function (the name stands for “least squares”), which you could call directly:

In [ ]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
theta_best_svd

This function computes $\widehat{\boldsymbol{\theta}}=\mathbf{X}^{+} \mathbf{y}$, where $X^+$
is the pseudoinverse of $X$, You can use np.linalg.pinv() to compute the pseudoin‐ verse directly:

### what is svd?

To compute the matrix Σ+, the algorithm takes Σ and sets to zero all values smaller than a tiny threshold value, then it replaces all the non-zero values with their inverse, and finally it transposes the resulting matrix. This approach is more efficient than computing the Normal Equation, plus it handles edge cases nicely: indeed, the Normal Equation may not work if the matrix XTX is not invertible (i.e., singular), such as if m < n or if some features are redundant, but the pseudoinverse is always defined.

In [ ]:
np.linalg.pinv(X_b).dot(y)

### Gradient descent

\begin{align*}
    \frac{\partial}{\partial \theta_{j}} \operatorname{MSE}(\boldsymbol{\theta}) & =\frac{2}{m} \sum_{i=1}^{m}\left(\boldsymbol{\theta}^{T} \mathbf{x}^{(i)}-y^{(i)}\right) x_{j}^{(i)}\\
    \nabla_{\boldsymbol{\theta}} \operatorname{MSE}(\boldsymbol{\theta}) &=\left(\begin{array}{c}
\frac{\partial}{\partial \theta_{0}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\frac{\partial}{\partial \theta_{1}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\vdots \\
\frac{\partial}{\partial \theta_{n}} \operatorname{MSE}(\boldsymbol{\theta})
\end{array}\right)=\frac{2}{m} \mathbf{X}^{T}(\mathbf{X} \boldsymbol{\theta}-\mathbf{y})
\end{align*}

In [ ]:
from sklearn.metrics import mean_squared_error
eta = 0.001 # learning rate
n_iterations = 1000
m = 100

plt.plot(X, y, 'bo',  markersize=3)
loss = []
theta = np.random.randn(2, 1)
for i in range(n_iterations):
    y_pred = X_b.dot(theta)
    if i % 100 == 0:
        plt.plot(X, y_pred, 'r-', linewidth=0.5)
    gradients = 2/m * X_b.T.dot(X_b.dot(theta)-y)
    theta = theta - eta * gradients
    loss.append(mean_squared_error(y, y_pred))
print(theta)

In [ ]:
plt.plot(np.arange(n_iterations), loss, 'b-')

### Stochastic gradient descent


In [ ]:
n_epochs = 50
t0, t1 = 5, 50 # learning schedule hyperparameters
def learning_schedule(t):
    return t0/(t+t1)
theta = np.random.randn(2,1) # random initialization
loss = []
for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        y_pred = X_b.dot(theta)
        loss.append(mean_squared_error(y, y_pred))

In [ ]:
plt.plot(np.arange(len(loss)), loss, 'b-')


In [ ]:
from sklearn.linear_model import SGDRegressor

# penalty=None: no regularization
# tol=1e-3: stop when loss is less than 1e-3
# eta0: learning rate

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X, y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_


### Polynomial regression


In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)
plt.plot(X, y, 'bo', markersize=3)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly_features.fit_transform(X)
X[0], X[0]**2, X_poly[0]

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.coef_, lin_reg.intercept_

In [ ]:
X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)
X_plot_poly = poly_features.fit_transform(X_plot)
print(X_plot_poly.shape)

In [ ]:
plt.plot(X, y, 'bo', markersize=3)
plt.plot(np.linspace(-3, 3, 100),lin_reg.predict(X_plot_poly), 'r-')

### Learning curve

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

In [ ]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)

In [ ]:
from sklearn.pipeline import Pipeline
polynomial_regression = Pipeline([
        ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
        ("lin_reg", LinearRegression()),
    ])
plot_learning_curves(polynomial_regression, X, y)



The Bias/Variance Tradeoff
An important theoretical result of statistics and Machine Learning is the fact that a model’s generalization error can be expressed as the sum of three very different errors:
1. Bias: This part of the generalization error is due to wrong assumptions, such as assum‐ ing that the data is linear when it is actually quadratic. A high-bias model is most likely to underfit the training data.
2. Variance: This part is due to the model’s excessive sensitivity to small variations in the training data. A model with many degrees of freedom (such as a high-degree pol‐ ynomial model) is likely to have high variance, and thus to overfit the training data.
3. Irreducible error: This part is due to the noisiness of the data itself. The only way to reduce this part of the error is to clean up the data (e.g., fix the data sources, such as broken sensors, or detect and remove outliers).
Increasing a model’s complexity will typically increase its variance and reduce its bias. Conversely, reducing a model’s complexity increases its bias and reduces its variance. This is why it is called a tradeoff.


In [ ]:
from sklearn.linear_model import Ridge

# two ways
poly_features = PolynomialFeatures(degree=40, include_bias=False)
X_poly = poly_features.fit_transform(X)
ridge_reg = Ridge(alpha=0, solver='cholesky')
ridge_reg.fit(X_poly, y)
print(ridge_reg.coef_, ridge_reg.intercept_)

# or add penalty
# sgd_reg = SGDRegressor(penalty='l2')
# sgd_reg.fit(X, y.ravel())

X_plot = poly_features.fit_transform(np.linspace(-3, 3, 100).reshape(-1, 1))
plt.plot(X, y, 'bo', markersize=3)
plt.plot(np.linspace(-3, 3, 100), ridge_reg.predict(X_plot), 'r-')

### Lasso regression

#### Why Lasso results in sparsity

Consider a vector $\vec{x}=(1, \varepsilon) \in \mathbb{R}^{2}$, where $\epsilon>0$
is small, then $l_1$ and $l_2$ norms of $\vec{x}$ are given by
$$\|\vec{x}\|_{1}=1+\varepsilon,\|\vec{x}\|_{2}^{2}=1+\varepsilon^{2}$$

After introducing some regularization procedure, we are going to reduce the
magnitude of one of the elements of $\vec{x}$ by $\delta<\epsilon$, if we change
$x_1$ to $1-\delta$, then
$$\|\vec{x}-(\delta, 0)\|_{1}=1-\delta+\varepsilon,\|\vec{x}-(\delta, 0)\|_{2}^{2}=1-2 \delta+\delta^{2}+\varepsilon^{2}$$

then reduction of norm are

\begin{align}
\Delta l_1 &=\delta\\
\Delta l_2 &=2\delta-\delta^2
\end{align}

On the other hand, if we reduce $x_2$, the smaller one by $\delta$,
$$\|\vec{x}-(0, \delta)\|_{1}=1-\delta+\varepsilon, \quad\|\vec{x}-(0, \delta)\|_{2}^{2}=1-2 \varepsilon \delta+\delta^{2}+\varepsilon^{2}$$

the reduction would be

\begin{align}
    \Delta l_1 &=\delta\\
    \Delta l_2 &=2\epsilon\delta-\delta^2
\end{align}

Thus, when penalizing a moedel using $l_2$ norm, it is highly likely that
anything will ever be set to zero, because $\epsilon$ is small and $\delta$ is even
smaller.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_features = PolynomialFeatures(degree=10)
X_poly = poly_features.fit_transform(X)
lasso_reg = Lasso(alpha=1)
lasso_reg.fit(X_poly, y)
print(lasso_reg.coef_, lasso_reg.intercept_)

### Logistic regression

The cost function is

\begin{equation}
J(\boldsymbol{\theta})=-\frac{1}{m} \sum_{i=1}^{m}\left[y^{(i)} \log \left(\hat{p}^{(i)}\right)+\left(1-y^{(i)}\right) \log \left(1-\hat{p}^{(i)}\right)\right]
\end{equation}

Derivatives is

$$\frac{\partial}{\partial \theta_{j}} \mathrm{~J}(\boldsymbol{\theta})=\frac{1}{m} \sum_{i=1}^{m}\left(\sigma\left(\boldsymbol{\theta}^{T} \mathbf{x}^{(i)}\right)-y^{(i)}\right) x_{j}^{(i)}$$

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

In [ ]:
iris['feature_names']

In [ ]:
iris['data'][:, 3:]

In [ ]:
X = iris['data'][:, 3:]
y = (iris['target'] == 2).astype(np.int)

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X, y)

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
plt.plot(X_new, y_proba[:, 1], "g-", label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", label="Not Iris-Virginica")

In [ ]:
print(y_proba)


### Softmax function

$$\hat{p}_{k}=\sigma(\mathbf{s}(\mathbf{x}))_{k}=\frac{\exp \left(s_{k}(\mathbf{x})\right)}{\sum_{j=1}^{K} \exp \left(s_{j}(\mathbf{x})\right)}$$

the prediction is

$$\hat{y}=\underset{k}{\operatorname{argmax}} \sigma(\mathbf{s}(\mathbf{x}))_{k}=\underset{k}{\operatorname{argmax}} s_{k}(\mathbf{x})=\underset{k}{\operatorname{argmax}}\left(\left(\boldsymbol{\theta}^{(k)}\right)^{T} \mathbf{x}\right)$$

the cross entropy loss is

$$J(\boldsymbol{\Theta})=-\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_{k}^{(i)} \log \left(\hat{p}_{k}^{(i)}\right)$$

Gradient vector for class $k$

$$\nabla_{\boldsymbol{\theta}^{(k)}} J(\boldsymbol{\Theta})=\frac{1}{m} \sum_{i=1}^{m}\left(\hat{p}_{k}^{(i)}-y_{k}^{(i)}\right) \mathbf{x}^{(i)}$$

In [ ]:
th#%%

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
X = 2 * np.random.rand(100, 1)
y = 4 + 3*X + np.random.rand(100, 1)

In [ ]:
plt.plot(X, y, 'bo',  markersize=3)

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]
print(X_b.shape)

In [ ]:
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

In [ ]:
y_pred = X_b.dot(theta_best)

In [ ]:
plt.plot(X, y, 'bo',  markersize=3)
plt.plot(X, y_pred, 'r-', linewidth=0.5)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

The LinearRegression class is based on the scipy.linalg.lstsq() function (the name stands for “least squares”), which you could call directly:

In [ ]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
theta_best_svd

This function computes $\widehat{\boldsymbol{\theta}}=\mathbf{X}^{+} \mathbf{y}$, where $X^+$
is the pseudoinverse of $X$, You can use np.linalg.pinv() to compute the pseudoin‐ verse directly:

### what is svd?

To compute the matrix Σ+, the algorithm takes Σ and sets to zero all values smaller than a tiny threshold value, then it replaces all the non-zero values with their inverse, and finally it transposes the resulting matrix. This approach is more efficient than computing the Normal Equation, plus it handles edge cases nicely: indeed, the Normal Equation may not work if the matrix XTX is not invertible (i.e., singular), such as if m < n or if some features are redundant, but the pseudoinverse is always defined.

In [ ]:
np.linalg.pinv(X_b).dot(y)

### Gradient descent

\begin{align*}
    \frac{\partial}{\partial \theta_{j}} \operatorname{MSE}(\boldsymbol{\theta}) & =\frac{2}{m} \sum_{i=1}^{m}\left(\boldsymbol{\theta}^{T} \mathbf{x}^{(i)}-y^{(i)}\right) x_{j}^{(i)}\\
    \nabla_{\boldsymbol{\theta}} \operatorname{MSE}(\boldsymbol{\theta}) &=\left(\begin{array}{c}
\frac{\partial}{\partial \theta_{0}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\frac{\partial}{\partial \theta_{1}} \operatorname{MSE}(\boldsymbol{\theta}) \\
\vdots \\
\frac{\partial}{\partial \theta_{n}} \operatorname{MSE}(\boldsymbol{\theta})
\end{array}\right)=\frac{2}{m} \mathbf{X}^{T}(\mathbf{X} \boldsymbol{\theta}-\mathbf{y})
\end{align*}

In [ ]:
from sklearn.metrics import mean_squared_error
eta = 0.001 # learning rate
n_iterations = 1000
m = 100

plt.plot(X, y, 'bo',  markersize=3)
loss = []
theta = np.random.randn(2, 1)
for i in range(n_iterations):
    y_pred = X_b.dot(theta)
    if i % 100 == 0:
        plt.plot(X, y_pred, 'r-', linewidth=0.5)
    gradients = 2/m * X_b.T.dot(X_b.dot(theta)-y)
    theta = theta - eta * gradients
    loss.append(mean_squared_error(y, y_pred))
print(theta)

In [ ]:
plt.plot(np.arange(n_iterations), loss, 'b-')

### Stochastic gradient descent


In [ ]:
n_epochs = 50
t0, t1 = 5, 50 # learning schedule hyperparameters
def learning_schedule(t):
    return t0/(t+t1)
theta = np.random.randn(2,1) # random initialization
loss = []
for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        y_pred = X_b.dot(theta)
        loss.append(mean_squared_error(y, y_pred))

In [ ]:
plt.plot(np.arange(len(loss)), loss, 'b-')


In [ ]:
from sklearn.linear_model import SGDRegressor

# penalty=None: no regularization
# tol=1e-3: stop when loss is less than 1e-3
# eta0: learning rate

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X, y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_


### Polynomial regression


In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)
plt.plot(X, y, 'bo', markersize=3)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly_features.fit_transform(X)
X[0], X[0]**2, X_poly[0]

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.coef_, lin_reg.intercept_

In [ ]:
X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)
X_plot_poly = poly_features.fit_transform(X_plot)
print(X_plot_poly.shape)

In [ ]:
plt.plot(X, y, 'bo', markersize=3)
plt.plot(np.linspace(-3, 3, 100),lin_reg.predict(X_plot_poly), 'r-')

### Learning curve

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

In [ ]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)

In [ ]:
from sklearn.pipeline import Pipeline
polynomial_regression = Pipeline([
        ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
        ("lin_reg", LinearRegression()),
    ])
plot_learning_curves(polynomial_regression, X, y)



The Bias/Variance Tradeoff
An important theoretical result of statistics and Machine Learning is the fact that a model’s generalization error can be expressed as the sum of three very different errors:
1. Bias: This part of the generalization error is due to wrong assumptions, such as assum‐ ing that the data is linear when it is actually quadratic. A high-bias model is most likely to underfit the training data.
2. Variance: This part is due to the model’s excessive sensitivity to small variations in the training data. A model with many degrees of freedom (such as a high-degree pol‐ ynomial model) is likely to have high variance, and thus to overfit the training data.
3. Irreducible error: This part is due to the noisiness of the data itself. The only way to reduce this part of the error is to clean up the data (e.g., fix the data sources, such as broken sensors, or detect and remove outliers).
Increasing a model’s complexity will typically increase its variance and reduce its bias. Conversely, reducing a model’s complexity increases its bias and reduces its variance. This is why it is called a tradeoff.


In [ ]:
from sklearn.linear_model import Ridge

# two ways
poly_features = PolynomialFeatures(degree=40, include_bias=False)
X_poly = poly_features.fit_transform(X)
ridge_reg = Ridge(alpha=0, solver='cholesky')
ridge_reg.fit(X_poly, y)
print(ridge_reg.coef_, ridge_reg.intercept_)

# or add penalty
# sgd_reg = SGDRegressor(penalty='l2')
# sgd_reg.fit(X, y.ravel())

X_plot = poly_features.fit_transform(np.linspace(-3, 3, 100).reshape(-1, 1))
plt.plot(X, y, 'bo', markersize=3)
plt.plot(np.linspace(-3, 3, 100), ridge_reg.predict(X_plot), 'r-')

### Lasso regression

#### Why Lasso results in sparsity

Consider a vector $\vec{x}=(1, \varepsilon) \in \mathbb{R}^{2}$, where $\epsilon>0$
is small, then $l_1$ and $l_2$ norms of $\vec{x}$ are given by
$$\|\vec{x}\|_{1}=1+\varepsilon,\|\vec{x}\|_{2}^{2}=1+\varepsilon^{2}$$

After introducing some regularization procedure, we are going to reduce the
magnitude of one of the elements of $\vec{x}$ by $\delta<\epsilon$, if we change
$x_1$ to $1-\delta$, then
$$\|\vec{x}-(\delta, 0)\|_{1}=1-\delta+\varepsilon,\|\vec{x}-(\delta, 0)\|_{2}^{2}=1-2 \delta+\delta^{2}+\varepsilon^{2}$$

then reduction of norm are

\begin{align}
\Delta l_1 &=\delta\\
\Delta l_2 &=2\delta-\delta^2
\end{align}

On the other hand, if we reduce $x_2$, the smaller one by $\delta$,
$$\|\vec{x}-(0, \delta)\|_{1}=1-\delta+\varepsilon, \quad\|\vec{x}-(0, \delta)\|_{2}^{2}=1-2 \varepsilon \delta+\delta^{2}+\varepsilon^{2}$$

the reduction would be

\begin{align}
    \Delta l_1 &=\delta\\
    \Delta l_2 &=2\epsilon\delta-\delta^2
\end{align}

Thus, when penalizing a moedel using $l_2$ norm, it is highly likely that
anything will ever be set to zero, because $\epsilon$ is small and $\delta$ is even
smaller.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_features = PolynomialFeatures(degree=10)
X_poly = poly_features.fit_transform(X)
lasso_reg = Lasso(alpha=1)
lasso_reg.fit(X_poly, y)
print(lasso_reg.coef_, lasso_reg.intercept_)

### Logistic regression

The cost function is

\begin{equation}
J(\boldsymbol{\theta})=-\frac{1}{m} \sum_{i=1}^{m}\left[y^{(i)} \log \left(\hat{p}^{(i)}\right)+\left(1-y^{(i)}\right) \log \left(1-\hat{p}^{(i)}\right)\right]
\end{equation}

Derivatives is

$$\frac{\partial}{\partial \theta_{j}} \mathrm{~J}(\boldsymbol{\theta})=\frac{1}{m} \sum_{i=1}^{m}\left(\sigma\left(\boldsymbol{\theta}^{T} \mathbf{x}^{(i)}\right)-y^{(i)}\right) x_{j}^{(i)}$$

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

In [ ]:
iris['feature_names']

In [ ]:
iris['data'][:, 3:]

In [ ]:
X = iris['data'][:, 3:]
y = (iris['target'] == 2).astype(np.int)

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X, y)

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
plt.plot(X_new, y_proba[:, 1], "g-", label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", label="Not Iris-Virginica")

In [ ]:
print(y_proba)


### Softmax function

$$\hat{p}_{k}=\sigma(\mathbf{s}(\mathbf{x}))_{k}=\frac{\exp \left(s_{k}(\mathbf{x})\right)}{\sum_{j=1}^{K} \exp \left(s_{j}(\mathbf{x})\right)}$$

the prediction is

$$\hat{y}=\underset{k}{\operatorname{argmax}} \sigma(\mathbf{s}(\mathbf{x}))_{k}=\underset{k}{\operatorname{argmax}} s_{k}(\mathbf{x})=\underset{k}{\operatorname{argmax}}\left(\left(\boldsymbol{\theta}^{(k)}\right)^{T} \mathbf{x}\right)$$

the cross entropy loss is

$$J(\boldsymbol{\Theta})=-\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_{k}^{(i)} \log \left(\hat{p}_{k}^{(i)}\right)$$

Gradient vector for class $k$

$$\nabla_{\boldsymbol{\theta}^{(k)}} J(\boldsymbol{\Theta})=\frac{1}{m} \sum_{i=1}^{m}\left(\hat{p}_{k}^{(i)}-y_{k}^{(i)}\right) \mathbf{x}^{(i)}$$

In [41]:
X = iris["data"][:, (2, 3)] # petal length, petal width
y = iris["target"]

In [42]:
softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
softmax_reg.fit(X, y)

LogisticRegression(C=10, multi_class='multinomial')